In [1]:
#   https://sly.readthedocs.io/en/latest/sly.html
from sly import Lexer, Parser

In [2]:
class CalcLexer(Lexer):
    tokens = {INSTANCIA, FIMACAO,ID, NUM, PLUS, TIMES, MINUS, DIVIDE, IGUAL, LPAREN, RPAREN,\
              INICIO,INTEIRO,IMPRIMA,LEIA,SE,ENTAO,SENAO,FIMSE,FIM,PARA,ATE,PASSO,FIMPARA,MENOR,MAIOR,STRING}
    
    ignore = '\t'

    # Tokens
    STRING = r'\".*\"'
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUM = r'\d+'
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    IGUAL = r'='
    LPAREN = r'\('
    RPAREN = r'\)'
    MENOR = r'\<'
    MAIOR = r'\>'
    FIMACAO = r'\;'
    INSTANCIA = r'\:'


    # Special cases (ex.: palavras reservadas)
    ID['inicio']   = INICIO
    ID['inteiro']  = INTEIRO
    ID['imprima']  = IMPRIMA
    ID['leia']     = LEIA
    ID['se']       = SE
    ID['entao']    = ENTAO
    ID['senao']    = SENAO
    ID['fim_se']   = FIMSE
    ID['fim']      = FIM
    ID['para']     = PARA
    ID['ate']      = ATE
    ID['passo']    = PASSO
    ID['fim_para'] = FIMPARA

    # Ignored pattern
    ignore_newline = r'\n+'

    # Extra action for newlines
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')
        
    # Ignored pattern
    ignore_space = r' '

    # Extra action for newlines
    def ignore_space(self, t):
        self.lineno += t.value.count(' ')

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [3]:
class CalcParser(Parser):
    tokens = CalcLexer.tokens
    
    precedence = (
        ('left',PLUS,MINUS),
        ('left',TIMES,DIVIDE)
    )
    
    def __init__(self):
        self.vars = {}
        print('\nInicializando Parser... \n')
        
#---------------------------------------------------------------        
        
    @_("INICIO comandos FIM")
    def initialD(self,p):
        print(p.comandos)
        return p.comandos    
    
#---------------------------------------------------------------

    @_("comando comandos")
    def comandos(self,p):
        return str(p.comando) + "\n" + p.comandos    
        
    @_("comando")
    def comandos(self,p):
        return str(p.comando)    
    
#---------------------------------------------------------------        

    @_("instanciacao")
    def comando(self,p):
        return p.instanciacao
    @_("atribuicao")
    def comando(self,p):
        return p.atribuicao
    @_("comandose")
    def comando(self,p):
        return p.comandose
    @_("comandopara")
    def comando(self,p):
        return p.comandopara
    @_("imprima")
    def comando(self,p):
        return p.imprima
    @_("leia")
    def comando(self,p):
        return p.leia       
    
#---------------------------------------------------------------
    
    @_("INTEIRO ID FIMACAO")
    def instanciacao(self,p):
        self.vars[p.ID] = '0'
        return p.ID + " = 0"
    @_("INTEIRO ID INSTANCIA expr_inteira_var FIMACAO")
    def instanciacao(self,p):
        self.vars[p.ID] = p.expr_inteira_var
        return p.ID + " = " + str(p.expr_inteira_var)
    
#---------------------------------------------------------------

    @_("ID INSTANCIA expr_inteira_var FIMACAO")
    def atribuicao(self,p):
        self.vars[p.ID] = p.expr_inteira_var
        return p.ID + " = " + str(p.expr_inteira_var)  

#---------------------------------------------------------------

    @_("SE comparacao ENTAO comandos FIMSE")
    def comandose(self,p):        
        if p.comparacao:
            return p.comandos
        else:
            pass
        return p.comparacao
    
    @_("SE comparacao ENTAO comandos SENAO comandos FIMSE")
    def comandose(self,p):
        if p.comparacao:
            return p.comandos0
        else:
            return p.comandos1
        
    #---------------------------------------------------------------

    @_("expr_inteira_var IGUAL expr_inteira_var")
    def comparacao(self,p):
        return p.expr_inteira_var0 == p.expr_inteira_var1
    
    @_("expr_inteira_var MAIOR expr_inteira_var")
    def comparacao(self,p):
        return p.expr_inteira_var0 > p.expr_inteira_var1
    
    @_("expr_inteira_var MENOR expr_inteira_var")
    def comparacao(self,p):
        return p.expr_inteira_var0 < p.expr_inteira_var1

#---------------------------------------------------------------        
    
    @_("PARA expr_inteira_var ATE expr_inteira_var PASSO expr_inteira_var comandos FIMPARA")
    def comandopara(self,p):
        string = ""
        for x in range(p.expr_inteira_var0,p.expr_inteira_var1,p.expr_inteira_var2):
            string += p.comandos + "\n"
        
        return string
        #return p.expr_inteira_var0  
    
#---------------------------------------------------------------        
    
    @_("IMPRIMA LPAREN STRING RPAREN FIMACAO")
    def imprima(self,p):
        #print(p.STRING)
        return p.STRING  
    @_("IMPRIMA LPAREN expr_inteira_var RPAREN FIMACAO")
    def imprima(self,p):
        #print(p.expr_inteira_var)
        return p.expr_inteira_var 
     
#---------------------------------------------------------------        
    
    @_("LEIA LPAREN ID RPAREN FIMACAO")
    def leia(self,p):
        a = input("Input (numerico): ")
        self.vars[p.ID] = int(a)
        print()
        return "leia -> "+p.ID +" = "+ a  

#---------------------------------------------------------------
    
    @_("ID")
    def expr_inteira_var(self,p):
        try:
            return self.vars[p.ID]  
        except:
            return p.ID
    @_("NUM")
    def expr_inteira_var(self,p):
        return int(p.NUM)
    @_("expr_inteira_var PLUS expr_inteira_var")
    def expr_inteira_var(self,p):
        return p.expr0 + p.expr1        
    @_("expr_inteira_var MINUS expr_inteira_var")
    def expr_inteira_var(self,p):
        return p.expr0 - p.expr1        
    @_("expr_inteira_var TIMES expr_inteira_var")
    def expr_inteira_var(self,p):
        return p.expr0 * p.expr1        
    @_("expr_inteira_var DIVIDE expr_inteira_var")
    def expr_inteira_var(self,p):
        return p.expr0 / p.expr1


In [5]:
diretorio = 'por.txt'
fp = open(diretorio,"r")
texto = fp.read()
fp.close

lexer = CalcLexer()

for tok in lexer.tokenize(texto):
        print('token=%r, lexema=%r' % (tok.type, tok.value))

        
parser = CalcParser()

result = parser.parse(lexer.tokenize(texto))
if result == None:
    print('Codigo não pertence à linguagem!')
else:
    print('Codigo correto!')

token='INICIO', lexema='inicio'
token='INTEIRO', lexema='inteiro'
token='ID', lexema='a'
token='FIMACAO', lexema=';'
token='IMPRIMA', lexema='imprima'
token='LPAREN', lexema='('
token='STRING', lexema='"digite o valor "'
token='RPAREN', lexema=')'
token='FIMACAO', lexema=';'
token='LEIA', lexema='leia'
token='LPAREN', lexema='('
token='ID', lexema='a'
token='RPAREN', lexema=')'
token='FIMACAO', lexema=';'
token='SE', lexema='se'
token='ID', lexema='a'
token='IGUAL', lexema='='
token='NUM', lexema='5'
token='ENTAO', lexema='entao'
token='IMPRIMA', lexema='imprima'
token='LPAREN', lexema='('
token='STRING', lexema='"igual a 5"'
token='RPAREN', lexema=')'
token='FIMACAO', lexema=';'
token='SENAO', lexema='senao'
token='IMPRIMA', lexema='imprima'
token='LPAREN', lexema='('
token='STRING', lexema='"diferente de 5"'
token='RPAREN', lexema=')'
token='FIMACAO', lexema=';'
token='FIMSE', lexema='fim_se'
token='PARA', lexema='para'
token='NUM', lexema='0'
token='ATE', lexema='ate'
token='NUM', l